In [1]:
#-*- coding: UTF-8 -*-
import urllib
import pprint
import time
from urllib.error import HTTPError
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

In [11]:
#網頁的編碼是GBK(中國區域編碼)


for page in range(0,10):
    for sub_page in range(0,10):
        novel_URL = "https://www.wenku8.net/novel/" +  str(page) + "/" + str(sub_page) + "/index.htm"
        try:
            response = urllib.request.urlopen(novel_URL)
            html = response.read()
            print("網址為 {0}".format(novel_URL))
            soup = BeautifulSoup(html, "html5lib")
            print("現在的目錄為 {0} 子目錄為 {1}".format(page, sub_page))
            print("===" * 10)
            print(soup.find(id = "title"))
            print("    {0}".format(soup.find(id = "info")))
            table = soup.find_all("table")
            for tr in table:
                print(type(tr))
                vcss_list = tr.select(".vcss"))
                print("===" * 10)
                
                    
                
            print("暫停1秒鐘")
            time.sleep(1)
        except HTTPError:
            print("網址 {0} 沒有小說".format(novel_URL))
            print("暫停1秒鐘")
            time.sleep(1)
            continue

網址 https://www.wenku8.net/novel/0/0/index.htm 沒有小說
暫停1秒鐘
網址為 https://www.wenku8.net/novel/0/1/index.htm
現在的目錄為 0 子目錄為 1
<div id="title">文学少女</div>
    <div id="info">作者：野村美月</div>
<class 'bs4.element.Tag'>
        [<td class="vcss" colspan="4">第一卷 渴望死亡的小丑</td>, <td class="vcss" colspan="4">第二卷 渴求真爱的幽灵</td>, <td class="vcss" colspan="4">第三卷 沉陷过往的愚者</td>, <td class="vcss" colspan="4">第四卷 背负污名的天使</td>, <td class="vcss" colspan="4">第五卷 悲怆恸哭的巡礼者</td>, <td class="vcss" colspan="4">第六卷 孕育月花的水妖</td>, <td class="vcss" colspan="4">第七卷 惠临神明的作家（上）</td>, <td class="vcss" colspan="4">第八卷 惠临神明的作家（下）</td>, <td class="vcss" colspan="4">番外篇 今天的点心</td>, <td class="vcss" colspan="4">文学少女的秘密书架</td>, <td class="vcss" colspan="4">外传一 见习生的初恋</td>, <td class="vcss" colspan="4">“文学少女”登上石像怪与笨蛋的阶梯</td>, <td class="vcss" colspan="4">恋爱插话集第一弹</td>, <td class="vcss" colspan="4">恋爱插话集第二弹</td>, <td class="vcss" colspan="4">外传二 见习生的伤心</td>, <td class="vcss" colspan="4">恋爱插话集第三弹</td>, <td class="vcss" colspan="4">外传三 见习

KeyboardInterrupt: 

In [28]:
type(ccss_table)

NameError: name 'ccss_table' is not defined

In [38]:
#網頁的編碼是GBK(中國區域編碼)
import json
title_dictionary_1 = {"小說系列":{"文學少女":{"作者":{"野村美月":{"卷數":{"第二卷":{"小說名稱":"序章 代替自我介紹的回憶—當時的我是家裏蹲" , "內文":"殺了她吧！他因暴風般的瘋狂渾身顫抖，下定了決心。沒錯，殺吧，殺了她吧！爲了不讓倒流的時光回到原本的地方，也爲了永遠把她綑綁在他的世界裏。抱著刀子的屍首、啜飲血液、啃食皮肉、枕著骨頭，睡在同一個棺材裏。她的眼睛、鼻子、嘴唇、皮膚、血肉、骨頭，全部——全部都是屬于他的。他的十只手指，緊緊地掐住她白皙似雪、寒冷似冰的脖子，以輕柔的聲音呢喃說道：“再見了，背叛者夏夜乃。”這裏是厭惡俗世者的天堂。雖然有些紳士會說著這種話，跑到鄉下去隱居。不過只是個國三生的我，卻也隱居在自己的房間裏。我在大白天就拉起窗簾，窩在床上裹著棉被，一邊叨念著「真希望太陽不要升起，希望明天永遠不要到來」，一邊緊抓著床單，把臉埋在枕頭裏，抽抽噎噎地哭泣著。日本明明有那麽多國三的男生，爲什麽只有我遭遇到這種事？我到底是做了什麽？我並不是厭惡俗世。我只是因爲第一次寫的小說偶然獲得了新人大獎，偶然成爲史上最年輕的得獎者，偶然取了「井上美羽」這個像女生的筆名罷了。但是那本小說卻出乎意料的暢銷，宣傳活動還說我是什麽「謎樣的天才美少女作家」——爲此，我失去了最重要的東西。我跟從小就很喜歡的一個特別的女孩，已經再也無法見面了。但是世人卻對十四歲的天才少女贊譽有加，擅自猜測著井上美羽的真正身分，出版社的人也不斷催促我快點寫出下一本作品。美羽已經遭遇到那種事了，爲什麽我還得繼續寫小說？拜托你們放過我吧，我不是什麽天才作家，更不是適合撐著白色洋傘的大家閨秀。我再也不要寫小說了！我全身冒著冷汗，指尖也變得麻痹，胸口仿佛被萬鈞之力壓迫得喘不過氣，因此我關上房門，緊閉雙眼，塞住耳朵，阻絕一切外來的資訊，假裝一切都不曾發生過。在門外發生的事全都是夢。門裏的世界才是現實，門外的世界全都是謊言。拜托不要有人打開門，不要走進房裏。如果打開門，謊言的世界就會變成怒濤般的現實朝我襲來，我會被那個世界給吞噬而窒息的。我一邊咬著被汗水浸濕的棉被，咬到牙龈幾乎出血，一邊打從心底深深盼望時間可以回到過去，盼望一切都能重新來過。就算只有幾個月也行，如果能回到過去就好了如果願望真能實現，我絕對不會再寫什麽小說，也絕對不會去參加新人獎。我可以繼續保持平凡國中生的身份，繼續待在美羽身邊，看著美羽的笑臉，聽美羽說些像是樹葉篩落的陽光一樣美麗的故事，爲美羽寫下的活潑文字而陶醉。只要這樣我就滿足了，也無須畏懼世界和他人，可以平穩幸福地活下去。我好想回到過去。好想重新來過。神啊，我請求你，讓我回到寫小說之前的時光吧！但是，無論我這個國三男生在陰暗的房裏再怎麽認真祈禱，這種投機的心願也不可能會實現。漫長嚴冬結束後，我蹒跚地爬出房間，去參加考試，成了高中生。然後，到了高中二年級的夏天我在只有兩名成員的文藝社裏，勤奮地書寫「文學少女」的點心。（注：家裏蹲，原文是指因爲社會適應不良而自我封閉，長期把自己關在房裏的青少年，又稱「隱蔽青年」或「蘭居族」。）"}}}}}}}
with open('09.json', 'w', encoding='utf-8') as fp:
    json.dump(title_dictionary_1, fp,ensure_ascii=False)
